In [1]:
%pip install -U transformers datasets evaluate trl --quiet
%pip install -U unsloth --quiet
%pip install -U python-dotenv --quiet
%pip install evaluate
%pip install prettytable
%pip install rouge-score bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     

In [2]:
import pandas as pd
import torch
from torch.utils.data import DataLoader

from datasets import Dataset
from unsloth import FastLanguageModel
from huggingface_hub import hf_hub_download

from tqdm import tqdm

import evaluate
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
from prettytable import PrettyTable

from dotenv import load_dotenv
import os


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [3]:
load_dotenv("./src/.env")
HF_TOKEN = os.environ["HF_TOKEN"]

In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-1.5B-Instruct", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    load_in_4bit = False,
    token=HF_TOKEN
)
tokenizer.padding_side = "left" # for batch processing
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2024.12.2: Fast Qwen2 patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA L4. Max memory: 21.951 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536, padding_idx=151665)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    

In [5]:
def load_tsv_dataset(file_path):
    """
    Load the TSV file containing reviews and responses.
    """
    df = pd.read_csv(
        hf_hub_download(repo_id="RichardNooooh/AmazonFeedback",
                        filename=file_path,
                        repo_type="dataset"),
        sep="\t"
    )
    df = df.drop(columns=["ASIN"]) # ignore ID
    return Dataset.from_pandas(df)

SYSTEM_PROMPT = (
    "You are a helpful assistant for a business. "
    + "You are given a set of Amazon reviews for a given item, one for each rating out of 5, "
    + "and tasked with providing actionable feedback to help improve this item. "
    + "Please format your response into concise sentences, one for each actionable feedback. "
    + "Place each feedback on a bulletpoint."
)
def preprocess_function(examples):
    texts, labels = examples["text"], examples["labels"]
    # BROKEN - apply_chat_template doesn't add generation prompt.
    formatted_data = []
    for text in texts:
      conversation = [
          {
              "role": "system",
              "content": SYSTEM_PROMPT
          },
          {
              "role": "user",
              "content": text
          }
      ]
      formatted_text = tokenizer.apply_chat_template(conversation, tokenize=False)

      formatted_data.append(formatted_text)
    return {"text": formatted_data, "labels": labels}


In [6]:
dataset = load_tsv_dataset("test/baseline.tsv")
dataset = dataset.map(preprocess_function, batched=True)

test_dataloader = DataLoader(dataset, shuffle=False, batch_size=8)

test/baseline.tsv:   0%|          | 0.00/5.28M [00:00<?, ?B/s]

Map:   0%|          | 0/1929 [00:00<?, ? examples/s]

In [7]:
def generate_predictions(test_dataloader, model, tokenizer):
    predictions, references = [], []
    for batch in tqdm(test_dataloader):
        batch_encoding = tokenizer(batch["text"], return_tensors="pt", padding=True).to("cuda")
        input_len = batch_encoding["input_ids"].shape[-1]

        with torch.no_grad():
            output = model.generate(**batch_encoding,
                                    max_new_tokens=256)
        generated_texts = tokenizer.batch_decode(output[:, input_len:], skip_special_tokens=True)

        cleaned_texts = [text.removeprefix("assistant:\n") for text in generated_texts] # remove "Assistant:\n"
        predictions.extend(cleaned_texts)
        references.extend(batch['labels'])

    return predictions, references

def get_evaluations(predictions, references):
    bleu_metric = evaluate.load("bleu")
    rouge_metric = evaluate.load("rouge")
    meteor_metric = evaluate.load("meteor")
    bertscore_metric = evaluate.load("bertscore")

    bleu_score = bleu_metric.compute(predictions=predictions, references=references)
    rouge_score = rouge_metric.compute(predictions=predictions, references=references)
    meteor_score = meteor_metric.compute(predictions=predictions, references=references)
    bert_score = bertscore_metric.compute(predictions=predictions, references=references, lang="en")

    print("BLEU Score:", bleu_score)
    print("ROUGE Score:", rouge_score)
    print("METEOR Score:", meteor_score)
    # print("BERT Score:", bert_score)

    df = pd.DataFrame(bert_score)
    print(df.describe())

    table = PrettyTable()
    table.field_names = ["Metric", "Score Details"]
    table.add_row(["BLEU", f"{bleu_score['bleu']:.4f}"])

    rouge_details = (
        f"ROUGE-1: {rouge_score['rouge1']:.4f}, "
        f"ROUGE-2: {rouge_score['rouge2']:.4f}, "
        f"ROUGE-L: {rouge_score['rougeL']:.4f}"
    )
    table.add_row(["ROUGE", rouge_details])
    table.add_row(["METEOR", f"{meteor_score['meteor']:.4f}"])

    print("Evaluation Metrics Report")
    print(table)


In [8]:
predictions, references = generate_predictions(test_dataloader, model, tokenizer)

100%|██████████| 242/242 [31:14<00:00,  7.74s/it]


In [9]:
df = pd.DataFrame({"references": references, "predictions": predictions})
df.to_csv("./raw_eval.tsv", sep="\t", index=False)

In [10]:
import gc

# del model
# del tokenizer

gc.collect()

torch.cuda.empty_cache()

In [11]:
get_evaluations(predictions, references)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BLEU Score: {'bleu': 0.01830789849244547, 'precisions': [0.3087889588230833, 0.05415986949429037, 0.012708406202634954, 0.0035156993027633658], 'brevity_penalty': 0.6226986871235761, 'length_ratio': 0.6785675999022311, 'translation_length': 158244, 'reference_length': 233203}
ROUGE Score: {'rouge1': 0.1997768713070932, 'rouge2': 0.03563623246264715, 'rougeL': 0.10691553852710975, 'rougeLsum': 0.17773854625643803}
METEOR Score: {'meteor': 0.1463484057715641}
         precision       recall           f1
count  1929.000000  1929.000000  1929.000000
mean      0.808123     0.839813     0.822504
std       0.090086     0.035986     0.064086
min       0.646560     0.766966     0.706870
25%       0.692055     0.796621     0.738984
50%       0.851478     0.854380     0.855841
75%       0.879143     0.868491     0.870945
max       0.926149     0.918801     0.919259
Evaluation Metrics Report
+--------+---------------------------------------------------+
| Metric |                   Score Details  